# <center>TITRE - Projet Python</center>

## Membres:
* Corentin Drezen - 40007337, TD2
* Nathanael NOM - N°ETUDIANT, TD1
* Irfane NOM - N°ETUDIANT, TD2

## Introduction:
On souhaite réaliser un jeu du sytle <i>shoot them up</i> en 2D à l'aide du module $\color{#B267E6}{\text{pygame}}$ avec le personnage principal qui suit la souris

<img src="SpaceInvaders-Gameplay.gif">


In [ ]:
import pygame

## Timers
Le jeu a besoin de gérer plusieurs delais afin de réaliser le rendu graphique et certaines mécaniques.

Les jeux pygame fonctionent avec une boucle `while` 'infinie' car le systeme d'évenements est uniquement utilisable avec une fonction `pygame.event.get()` qui retourne une liste d'évenements.

In [ ]:
def main():
    #...
    while leJeuTourne:
        #...
        for event in pygame.event.get():
            #...

Le programme va s'éxécuter aussi vite qu'il peut mais un écran ne peut afficher que 60 à 144 images/secondes aujourd'hui et les appels au fonctions graphique incéssantes pouraient ralentir le jeu. Donc on a besoin d'un délai mais on ne veut pas bloquer le programme.

Alors on utilise les timers intégrés dans le module pygame qui fonctionent en parallèle et peuvents être utilisé avec son sytème d'évenements

<br>
On commence par définir les identifiants de nos propres évenements:

In [ ]:
AFFICHAGE_TIMER_EVENT = pygame.USEREVENT + 0 #évenement pour l'affichage
DELAY_TIRE_EVENT = pygame.USEREVENT + 2 #évenement pour avoir un délai entre chaque tir du personnage
SPAWN_TIMER_EVENT = pygame.USEREVENT + 3 #évenment pour l'apparition des ennemis

D'après le wiki il faut que ce soit des `int` compris entre `pygame.USEREVENT` (=24) et `pygame.NUMEVENTS - 1` (=32). - *"User defined events should have a value in the inclusive range of USEREVENT to NUMEVENTS - 1. It is recommended all user events follow this system."*

On doit ensuite appeler `pygame.time.set_timer(`***identifiant*** (int)`,` ***délai*** (int)`,` optionel: ***une_fois***=False`)` là où on souhaite lancer le timer. Elle déclanchera un évenement après chaque 'pauses'.

In [ ]:
def main():
    #...
    pygame.time.set_timer(AFFICHAGE_TIMER_EVENT, int(1000/fps)) 

Ici on utlise `int(1000/`**fps**`)` comme valeur pour le délai afin d'avoir une valeur relative à 'fps' (pour frames par secondes). Si 'fps' est égal à 60 on obtient 60 évenements `AFFICHAGE_TIMER_EVENT` par secondes: 1 toutes les ~17 ms.

Dans la boucle qui itère parmis tout les évenements on 'filtre' notre `AFFICHAGE_TIMER_EVENT` et on peut ainsi appelé notre fonction d'affichage/dessin/'peinture' à intervale régulier.

In [ ]:
#...
while leJeuTourne:
    #...
    for event in pygame.event.get():
        
        if event.type == AFFICHAGE_TIMER_EVENT:
            #...changer quelque chose juste avant le rafraichisement..
            Affichage()

## Affichage

## Déplacements

Les déplacements des acteurs du jeu son gérés dans leur `class` (Nafaire) avec la fonction `déplacement(`***x*** (int)`,` ***y*** (int)`)`.

`class` Nafaire:


In [ ]:
class Nafaire:
    def __init__(self, position=(0,0), animation=None, (...), vitesseX=0.1, vitesseY=0.1, rect=None):
        
        self.x , self.y = position
        self.anciennePos = position
        self.animation = animation
        #...   
        self.vitesseX = vitesseX
        self.vitesseY = vitesseY    
        #...

    def deplacement(self, x, y):
        #...

    def detectCollisions(self):
        #...

Cette `class` a plusieurs propriétés et arguments:
- `position` (tuple), de simples coordonées dans la fenêtre du jeu. Elles sont ensuite distribuées dans les propriétés ***x*** et ***y***
- `anciennePos` (tuple), elle sert à garder l'ancienne position en mémoire après un déplacement. Elle a la même valeur que `position` à l'initialisation de l'objet.
- `vitesseX` (float) et `vitesseY` (float), ce ne sont pas vraiment des vitesses de physique '*v = d/t*' mais des multiplicateurs qui permettent un déplacement plus ou moins conséquents.
- `rect` (pygame.Rect), un type d'objet qui sert a dessiner une image rectangulaire avec les propriétés comme ***haut***, ***bas***, ***gauche***, ***droite*** ou encore ***centre*** avec pygame.
- `animation` (list), une liste de pygame.Surface (images), chaque frames d'animation de l'acteur. Leurs dimentions doivent être constantes.

Ces valeurs sont utilisées dans la fonction `déplacement(`***x*** (int)`,` ***y*** (int)`)`:

In [ ]:
def deplacement(self, x, y):
        self.anciennePos = (self.x, self.y)
        self.x += x * self.vitesseX
        self.y += y * self.vitesseY
        self.rect = self.animation[0].get_rect(center=(self.x, self.y))

        self.detectCollisions()

A chaque fois que la fonction est appelée les arguments ***x*** et ***y***, qui peuvent avoir des valeurs négatives, sont multipliés par leurs 'vitesses' respectives puis ajoutés aux propriétés ***self.x*** et ***self.y***. Ce qui donne l'illusion de mouvement car cette fonction est appelée très régulièrement et met à jour les valeurs utilisées par la fonction d'affichage. 

La propriété ***rect*** est mis à jour avec la fonction `get_rect()` de l'objet `pygame.Surface` (image dans pygame). C'est elle qui servira à placer l'acteur lors de l'affichage car ce type d'objet est plus pratique que de simples coordonnées. Sa propriété ***centre*** permettra de placer l'acteur sur un point à partir de son centre facilement au lieu le placer à partir de son 'haut gauche'.

L'ancienne position est conservé lors du déplacement car elle servira à gérer les collisions.

La fonction `.detectCollisions()` qui sert a trouver des collisions est appelée ici car c'est seulement après un déplacement qu'il peut y avoir une collisions avec un autre acteur.

<br>
Les déplacement du personnage sont effectué lorsque l'utlisateur appuie, relache, ou reste appuyé sur les touches Z, Q, S, et D.

Pour détécter ces entrés au clavier on se sert les évenement pygame:
 - `KEYDOWN`, qui permet de savoir quand une touche est enfoncée
 - `KEYUP`, qui permet de savoir quand une touche est relaché

 Afin de recevoir plusieurs évenement `KEYDOWN` à intervale régulier pendant qu'une touche reste enfoncée avec pygame on doit utiliser la fonction `pygame.key.set_repeat(`***délai*** (int)`,` ...`)` au début du programme qui peut prendre un ou plusieurs arguments dont le délai que l'on souhaite avoir entre chaque évenement `KEYDOWN`.

In [ ]:
#...
def main():  
    #...

    pygame.key.set_repeat(int(1000/fps))#permet d'avoir des evenements pour les touches qui restent appuyées, avec un délai pour éviter de bloquer le programme
    
    keysDown = None

    while leJeuTourne:

        ###gestion des evenements:
        for event in pygame.event.get():

            #...gestion d'autres évenements...

            elif event.type == pygame.KEYDOWN or event.type == pygame.KEYUP:
                keysDown = pygame.key.get_pressed()

            #...gestion d'autres évenements...

On utilise la fonction pygame `pygame.key.get_pressed()` qui retourne une liste des touches enfoncées au moment où elle est appelée. Cette liste sera utlisé au moment de l'affichage (cf Timers) car on ne veut pas de déplacements constants qui bloquerai le programme et qui ne serai visibles qu'au moment du rafraichissement au final.

On déplace le personnage au moment de l'affichage si des touches sont enfoncés.

In [ ]:
    #...
    while leJeuTourne:

        ###gestion des evenements:
        for event in pygame.event.get():

            #...gestion d'autres évenements...
            
            elif event.type == AFFICHAGE_TIMER_EVENT:
            
                #gestion des déplacement à l'évenement d'affichage pour eviter de bloquer l'affichage ou la file d'évenements avec des déplacements constants
                if(keysDown != None):

                    tmpDeplacementJoueur = dim(0, 0)
       
                    if keysDown[pygame.K_z]:             
                        tmpDeplacementJoueur.y -= 1
                    if keysDown[pygame.K_s]:
                        tmpDeplacementJoueur.y += 1
                    if keysDown[pygame.K_q]:
                        tmpDeplacementJoueur.x -= 1
                    if keysDown[pygame.K_d]:                
                        tmpDeplacementJoueur.x += 1  
                    ###...autres touches
                
                    if(tmpDeplacementJoueur.x != 0 or tmpDeplacementJoueur.y != 0):
                        Joueur.deplacement(tmpDeplacementJoueur.x, tmpDeplacementJoueur.y)

                    keysDown = None

                #...

                Affichage()

## Colisions
Afin d'empécher que les acteurs du jeu se traversent et que le joueur puisse sortir de la scène il faut un système de collision.

pygame ne gère pas les collisions mais le type `pygame.Rect` possède une fonction `.colliderect(`***rect*** (pygame.Rect)`)` qui retourne un `bool` qui permet de savoir si un rectangle est superposé sur un autre rectangle ou non.

Suivant les types d'acteur (`Nafaire`) on va chercher à détecter des collisions avec différent type d'autre acteur.
Pour différencier les type d'acteur et que l'objet soit 'conscient' de son 'type' on va utiliser une nouvelle propriété `self.type` comparable avec un enum. La class `enum` est présente depuis la version 3.4 de python.

Puis pour avoir une meilleur lisibilité et pour pouvoir agir en fonction des collisions entre différent type d'acteur dans le code pricipale on va définir un nouvelle évenement pygame `COLLISION_EVENT` qui sera déclenchable avec la fonction `pygame.event.post(`***event*** (pygame.event.Event)`)`.

In [ ]:
from enum import Enum
#...
COLLISION_EVENT = pygame.USEREVENT + 1 # définition de l'identifiant d'évenement des collisions
#...
class NafaireTypes(Enum):
    DEFAULT = 0 # valeur par défault utilisé par la class Nafaire si on ne lui donne pas de type
    JOUEUR = 1
    ENNEMI = 2
    BALLE = 3


Dans la fonction `Nafaire.detectCollisions()` on va tout d'abord tester si l'acteur est parti en dehors de la scene peut importe le type car on va vouloir détruire ou replacer l'objet si il n'est plus visible. Pour ça on compare ses coordonnées aux dimentions de la scène.



In [ ]:
def detectCollisions(self):

        if(self.x < 0 or self.x > dimentions.x or self.y < 0 or self.y > dimentions.y):
            #...

Si l'une d'elle est hors champs on déclenche notre évenement `COLLISION_EVENT` avec comme argument ***source***: *l'acteur lui même (`self` (`Nafaire`))* qui trouve la collision, et ***collision***: *rien* (`None`) (un `Nafaire` aussi en temps normal)

Un objet de type `pygame.event.Event` peut être initialisé avec plusieurs arguments personalisés. On l'initialise ici ainsi: `pygame.event.Event(`***identifiant*** (int)`,` ***argument*** (object)`,` ***argument1*** (object)`)`. Puis on l'envoie dans la file d'évenement avec `pygame.event.post()`. L'évenement pourra ainsi être traité dans la boucle principale.

In [ ]:
pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=None))

On va ensuite éffectuer une sorte de switch (pas encore présent en python) suivant le type d'acteur. On n'arrête pas la fonction si il y a une collision avec la bordure car il peut aussi y avoir une collision avec un acteur en même temps.

In [ ]:
def detectCollisions(self):

        if(self.x < 0 or self.x > dimentions.x or self.y < 0 or self.y > dimentions.y):
        #...

        if(self.type == NafaireTypes.JOUEUR):
            #...
        elif(self.type == NafaireTypes.ENNEMI):
            #...
        elif(self.type == NafaireTypes.BALLE):
            #...

Si on cherche des collisions depuis un acteur de type `JOUEUR` on s'intéresse uniquement au collisions avec les ennemies et le cadre. Donc on va itérer dans tout les ennemies et tester si le rectangle du joueur est en parti inclus dans un rectangle des ennemies.
Si le test est vrai on envoie un evenement `COLLISION_EVENT` avec comme arguments `source=self` (joueur (`Nafaire`)) et ` collision=enemi` (`Nafaire`).

In [ ]:
#...
if(self.type == NafaireTypes.JOUEUR):
               
    for enemi in enemies:
        if self.rect.colliderect(enemi.rect): 
            pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=enemi))

On ferra de même pour les acteurs de type `BALLE` sauf que l'on utilisera un `break` pour briser l'itération car on ne veut pas qu'elles touchent plusieurs enemies en même temps.

Si l'acteur est de type `ENNEMI` on va simplement tester si il rentre en collision avec le joueur. Donc on a juste à rajouter un `and` dans le `if` puis envoyer l'évenement si le test est vrai. Si l'acteur n'est pas de type `ENNEMI` le `and` ne sera pas executé:

In [ ]:
elif(self.type == NafaireTypes.ENNEMI and self.rect.colliderect(Joueur.rect)): 
    pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=Joueur))

Voici le code complet de la fonction:

In [ ]:
def detectCollisions(self):

        if(self.x < 0 or self.x > dimentions.x or self.y < 0 or self.y > dimentions.y): #si l'acteur part hors champs
            pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=None))        

        if(self.type == NafaireTypes.JOUEUR):
            
            for enemi in enemies:
                if self.rect.colliderect(enemi.rect): 
                    pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=enemi))

        elif(self.type == NafaireTypes.ENNEMI and self.rect.colliderect(Joueur.rect)): 
            pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=Joueur))

        elif(self.type == NafaireTypes.BALLE):
            
            for enemi in enemies:
                
                if self.rect.colliderect(enemi.rect): 
                    pygame.event.post(pygame.event.Event(COLLISION_EVENT, source=self, collision=enemi))
                    break


Les collisions détectées sont ensuite gérés dans la boucle principale du jeu (grâce au évenements) en fonction des types des deux entités impliqués dans la collision.

In [ ]:
while leJeuTourne:
#...
    for event in pygame.event.get():
        
        #...gestion d'autres évenements...
        
        elif event.type == COLLISION_EVENT:

            if(event.collision == None):#collision avec la 'bordure du cadre'
                #...               

            elif(event.source.type == NafaireTypes.BALLE and event.collision.type == NafaireTypes.ENNEMI):#collision balle ennemi
                #...

            elif(event.source.type == NafaireTypes.ENNEMI and event.collision.type == NafaireTypes.JOUEUR) or (event.source.type == NafaireTypes.JOUEUR and event.collision.type == NafaireTypes.ENNEMI):#collision joueur ennemi
                #...

Si c'est une collision avec la parti non visible de la scène et que l'acteur est un `JOUEUR` ou un `ENNEMI` on remplace ses coordonnées génantes par celle du cadre de la scène. Si c'est une balle on la détruit car on n'en a plus besoins et qu'une quantité impotante de balles hors champs pourais gravement réduire les performances.

In [ ]:
#...
elif event.type == COLLISION_EVENT:
    
    if(event.collision == None):#collision avec la 'bordure du cadre'
        if(event.source.type == NafaireTypes.JOUEUR or event.source.type == NafaireTypes.ENNEMI):
            ###corrige la position du joueur ou de l'ennemi pour qu'il ne sorte pas du cadre
            if event.source.x < 0 : event.source.x = 0
            if event.source.x > dimentions.x : event.source.x = dimentions.X
            if event.source.y < 0 : event.source.y = 0
            if event.source.y > dimentions.y : event.source.y = dimentions.y

        elif(event.source.type == NafaireTypes.BALLE):
            ###détruit les balles hors champs
            if event.source in balleList:
                balleList.remove(event.source)                


Si c'est une collision entre une `BALLE` et un `ENNEMI` on supprime la balle pour qu'elle ne traverse pas les ennemies et fait prendre des dégats à l'ennemi ou on le supprime tout simplement. Puis on incrémente le score.

In [ ]:
elif(event.source.type == NafaireTypes.BALLE and event.collision.type == NafaireTypes.ENNEMI):#collision balle ennemi
    if event.collision in enemies:
        enemies.remove(event.collision)
    if event.source in balleList:
        balleList.remove(event.source)
        score += 1

On cherche une 'source' de collision de type `BALLE` ici car seul les `BALLE`s recherchent les collisions avec les ennemis

La dernière collision traité est celle entre un `JOUEUR` et un `ENNEMI` peut importe le qui détecte la collision.
On remet alors la 'source' de la collision à son ancienne position afin d'empêcher qu'ils se traversent et on met à jour son rectangle.

Voici la partie complète de la gestion des collision dans la boucle pricipale:

In [ ]:
#...
for event in pygame.event.get():

            #...gestion d'autres évenements...
            
            elif event.type == COLLISION_EVENT:

                if(event.collision == None):#collision avec la 'bordure du cadre'
                
                    if(event.source.type == NafaireTypes.JOUEUR or event.source.type == NafaireTypes.ENNEMI):
                        ###corrige la position du joueur ou de l'ennemi pour qu'il ne sorte pas du cadre
                        if event.source.x < 0 : event.source.x = 0
                        if event.source.x > dimentions.x : event.source.x = dimentions.X
                        if event.source.y < 0 : event.source.y = 0
                        if event.source.y > dimentions.y : event.source.y = dimentions.y

                    elif(event.source.type == NafaireTypes.BALLE):
                        ###détruit les balles hors champs
                        if event.source in balleList:
                            balleList.remove(event.source)                

                elif(event.source.type == NafaireTypes.BALLE and event.collision.type == NafaireTypes.ENNEMI):#collision balle ennemi
                    if event.collision in enemies:
                        enemies.remove(event.collision)
                    if event.source in balleList:
                        balleList.remove(event.source)
                    score += 1
                
                elif(event.source.type == NafaireTypes.ENNEMI and event.collision.type == NafaireTypes.JOUEUR) or (event.source.type == NafaireTypes.JOUEUR and event.collision.type == NafaireTypes.ENNEMI):#collision joueur ennemi
                    event.source.x, event.source.y = event.source.anciennePos
                    event.source.rect = event.source.animation[0].get_rect(center=(event.source.x, event.source.y))

            #...gestion d'autres évenements...

## Tires

 La fonctionalité de tire permet à l'utilisateur de faire apparaitre des balles qui se déplacent à partir du haut du personnage quand il clique ou reste appuyé sur espace.
 
 Elle est répartie sur plusieurs parties du code.

 
 Pour détecter les cliques et les entrés au clavier on utilise plusieurs évenement pygame:
 - `KEYDOWN`, qui permet de savoir quand une touche est enfoncée.
 - `KEYUP`, qui permet de savoir quand une touche est relaché.
 - `MOUSEBUTTONUP`, qui permet de savoir quand un clique est relaché.
<br>

Comme vu précedement dans la partie 'Tires' pour recevoir plusieurs évenement `KEYDOWN` à intervale régulier pendant qu'une touche reste enfoncée avec pygame on doit utiliser la fonction `pygame.key.set_repeat(`***délai*** (int)`,` ...`)` au début du programme.

On crée une variable globale booléenne `peutTirer` et `tireParSecondes` qui servirons de verrou pour le 'fusil' avec l'évenement `DELAY_TIRE_EVENT`.
On défini également la constante `LEFT_CLICK` qui servira a différencier un clique gauche d'un clique droit. Ainsi que la variable `keysDown` qui sera une liste de touches.
 
On a aussi une variable `balleList` qui permettra d'itérer entre les objet 'balles' crées pour leurs déplacement et leurs interactions.

In [ ]:
    #...
    tireParSecondes = 8
    peutTirer = True
    DELAY_TIRE_EVENT = pygame.USEREVENT + 2
    #...
    def main():  
        #...
        global balleList
        #...
        pygame.key.set_repeat(int(1000/fps))#permet d'avoir des evenements pour les touches qui restent appuyées, avec un délai pour éviter de bloquer le programme
    
        LEFT_CLICK = 1
        keysDown = None

        while leJeuTourne:

            ###gestion des evenements:
            for event in pygame.event.get():

                #...gestion d'autres évenements...

                elif event.type == pygame.KEYDOWN or event.type == pygame.KEYUP:
                    #...
                    
                elif event.type == pygame.MOUSEBUTTONUP: # quand on clique avec la souris
                    #...

                #...gestion d'autres évenements...

On utilise la fonction pygame `pygame.key.get_pressed()` qui retourne une liste des touches enfoncées au moment où elle est appelée. Cette liste sera utlisé au moment de l'affichage car on ne veut pas de tirs ou de déplacements constants qui bloquerai le programme et qui ne serai visibles qu'au moment du rafraichissement au final.

On déplace chaque balles vers le haut au moment de l'affichage.

La fonction `tire()` sera appelée si les bonne touches sont utilisées et seulement si `peutTirer` est `True` pour pouvoir controler la fréquence de tir.

In [ ]:
###gestion des evenements:
for event in pygame.event.get():

    #...gestion d'autres évenements...

    elif event.type == pygame.KEYDOWN or event.type == pygame.KEYUP:
        keysDown = pygame.key.get_pressed()

        elif event.type == pygame.MOUSEBUTTONUP: #tire quand on clique avec la souris
            if (event.button == LEFT_CLICK and peutTirer == True): 
                tire()
    #...gestion d'autres évenements...

    elif event.type == AFFICHAGE_TIMER_EVENT:
            
        if(keysDown != None):

            #...gestion des touches de déplacement...

            if (keysDown[pygame.K_SPACE] and peutTirer == True):
                tire()           
                
            keysDown = None
            
            #...
        
        for b in balleList:
            b.deplacement(0, -1)

        #...

        Affichage()

La fonction tire lancée, elle verouillera le tir avec `peutTirer = False` et créera un objet `Nafaire` de type `BALLE` en haut du personnage avec une vitesse choisie.
Cet objet est ajouté à une liste qui sera utilisé pour déplacer les balles au moment du rafraîchissement.

On appelle ensuite la fonction `pygame.time.set_timer(`***identifiant*** (int)`,` ***délai*** (int)`,` optionel: ***une_fois***=False`)` (voir: Timers) comme vu précedement mais avec l'argument `once=True`.

Cela nous permet de déclanché un évennement `DELAY_TIRE_EVENT` au bout d'un délai seulement une seul fois.

In [ ]:
def tire():
    global peutTirer
    peutTirer = False
    balle = Nafaire([Joueur.rect.centerx + 4, Joueur.rect.top - 22], balleImg, type=NafaireTypes.BALLE, vitesseY=8)
    balleList.append(balle)

    pygame.time.set_timer(DELAY_TIRE_EVENT, int(1000/tireParSecondes), True)


Dans la gestions des évenements on traite `DELAY_TIRE_EVENT` par une affectation `True` à la variable `peutTirer` afin 'd'enlever le verrou'.

In [ ]:
###gestion des evenements:
        for event in pygame.event.get():

            #...gestion d'autres évenements...

            elif event.type == DELAY_TIRE_EVENT:
                peutTirer = True

            #...gestion d'autres évenements...